In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import shutil
import subprocess
from tqdm.notebook import tqdm
import time

import cv2
import numpy as np

import torch
from torch import nn

%matplotlib inline
from matplotlib import pylab as plt

import face_expression
from face_expression import utils

In [ ]:
sys.path.append("/Vol0/user/k.iskakov/dev/pykinect")
from pose_dataset_example import *

# # renat
# sys.path.append("/home/renat/workdir/git/kavatar_realtime/extern/pykinect/")
# from pose_dataset_example import *

In [ ]:
# input
op_root = "/Vol1/dbstore/datasets/k.iskakov/azure_people_test/openpose_vakhitov_format"
smplx_root = "/Vol1/dbstore/datasets/a.vakhitov/kinect_dataset/full_testcap_30"

# setup dirs for result
result_root_dir = "/Vol1/dbstore/datasets/k.iskakov/projects/face_expression"
result_dir = os.path.join(result_root_dir, "artifacts", "azure_people_video_inferer")

frame_dir = os.path.join(result_dir, "frames")
output_video_path = os.path.join(result_dir, "video.mp4")

shutil.rmtree(output_video_path, ignore_errors=True)
shutil.rmtree(frame_dir, ignore_errors=True)
os.makedirs(frame_dir, exist_ok=True)

# # input renat
# op_root = "/home/renat/workdir/data/kavatar/face_expression/openpose_vakhitov_format"
# smplx_root = "/home/renat/workdir/data/kavatar/face_expression/full_testcap_30"

# # setup dirs for result
# result_root_dir = "/home/renat/workdir/data/kavatar/face_expression/result"
# result_dir = os.path.join(result_root_dir, "artifacts", "azure_people_video_inferer")

# frame_dir = os.path.join(result_dir, "frames")
# output_video_path = os.path.join(result_dir, "video.mp4")

# shutil.rmtree(output_video_path, ignore_errors=True)
# shutil.rmtree(frame_dir, ignore_errors=True)
# os.makedirs(frame_dir, exist_ok=True)

In [ ]:
# vis
def load_joints_poses_dict(root_folder_bt, pid_lbl, dev_lbl):
    joints_json_path = root_folder_bt + '/' + pid_lbl + '/bt.json'
    with open(joints_json_path) as json_file:
        joints_poses_list = json.load(json_file)

    joints_poses_dict = dict()
    for i in tqdm(range(len(joints_poses_list))):
        try:
            joints_poses, joints_mask = get_kinect_joints(joints_poses_list, i, dev_lbl)
            joints_poses_dict[joints_poses_list[i]['frame_index']] = joints_poses
        except Exception as e:
            pass
            
    return joints_poses_dict

In [ ]:
root_folder_bt = "/Vol0/user/r.bashirov/workdir/git/data/smplx_kinect/test_capture/offline_processor2"
root_folder_rgb = "/Vol0/user/r.bashirov/workdir/git/data/smplx_kinect/test_capture/offline_processor2"

pk_root = "/Vol1/dbstore/datasets/a.vakhitov/projects/pykinect_fresh/pykinect/data/fit_realtime_data"
vposer_ckpt = "/Vol1/dbstore/datasets/a.vakhitov/projects/pykinect_fresh/smplify-x/smplify-x-data/vposer_v1_0"

## renat
# root_folder_bt = "/home/renat/workdir/data/kavatar/face_expression/offline_processor2"
# root_folder_rgb = root_folder_bt

# # pk_root = "/Vol1/dbstore/datasets/a.vakhitov/projects/pykinect_fresh/pykinect/data/fit_realtime_data"
# # pk_root = "/mnt/hdd10/kavatar/release_data/2020_09_09/pykinect"
# pk_root = "/home/renat/workdir/data/kavatar/face_expression/pk_root"
# vposer_ckpt = "/home/renat/workdir/data/kavatar/face_expression/vposer_v1_0"

fit_type = 'joints_op_face_hand'

start_index, n_frames, step = 10, 700, 2
frame_ids = np.arange(start_index, start_index + n_frames, step)

fit_dataset_path = smplx_root

dev_lbl = '000062692912'
pid_lbl = '04_simple'

In [ ]:
def cam_params_decode(cam_params):
    K_d_rgb = np.eye(3)
    K_d_rgb[0, 0] = cam_params['rgb_intrinsics']['fx']
    K_d_rgb[1, 1] = cam_params['rgb_intrinsics']['fy']
    K_d_rgb[0, 2] = cam_params['rgb_intrinsics']['cx']
    K_d_rgb[1, 2] = cam_params['rgb_intrinsics']['cy']

    K_rgb = np.eye(3)
    K_rgb[0, 0] = cam_params['rgb_undistorted_intrinsics']['fx']
    K_rgb[1, 1] = cam_params['rgb_undistorted_intrinsics']['fy']
    K_rgb[0, 2] = cam_params['rgb_undistorted_intrinsics']['cx']
    K_rgb[1, 2] = cam_params['rgb_undistorted_intrinsics']['cy']

    d_rgb = np.zeros(8)
    d_rgb[0] = cam_params['rgb_intrinsics']['k1']
    d_rgb[1] = cam_params['rgb_intrinsics']['k2']
    d_rgb[2] = cam_params['rgb_intrinsics']['p1']
    d_rgb[3] = cam_params['rgb_intrinsics']['p2']
    d_rgb[4] = cam_params['rgb_intrinsics']['k3']
    d_rgb[5] = cam_params['rgb_intrinsics']['k4']
    d_rgb[6] = cam_params['rgb_intrinsics']['k5']
    d_rgb[7] = cam_params['rgb_intrinsics']['k6']

    T_depth_to_rgb = np.eye(4)
    rvec = np.zeros(3)
    tvec = np.zeros(3)
    for i in range(0, 3):
        rvec[i] = cam_params['depth_to_rgb']['r'][i]
        tvec[i] = cam_params['depth_to_rgb']['t'][i]

    tvec /= 1000

    R, jac = cv2.Rodrigues(rvec)
    T_depth_to_rgb[0:3, 0:3] = R
    T_depth_to_rgb[0:3, 3] = tvec

    return K_d_rgb, d_rgb, K_rgb, T_depth_to_rgb

# load device calibs
mkv_meta_json_path = root_folder_bt + '/' + pid_lbl + '/' + dev_lbl + '/mkv_meta.json'
with open(mkv_meta_json_path) as f:
    cam_params = json.load(f)

K_d_rgb, d_rgb, K_rgb, T_depth_to_rgb = cam_params_decode(cam_params)
K_d_rgb = K_d_rgb
d_rgb = d_rgb
rvec, jac = cv2.Rodrigues(T_depth_to_rgb[0:3, 0:3])
tvec = T_depth_to_rgb[0:3, 3]
rvec = rvec
tvec = tvec

# if not os.path.exists(mkv_meta_json_path):
#     print('dev not found')
# K_rgb, K_d, T_depth_to_rgb = read_device_calibs(mkv_meta_json_path)

joints_poses_dict = load_joints_poses_dict(root_folder_bt, pid_lbl, dev_lbl)

In [ ]:
# load inferer
# import lib
# FACE_EXPRESSION_SRC_ROOT = "/home/renat/Desktop/face_expression"
# sys.path.append(FACE_EXPRESSION_SRC_ROOT)
from face_expression import Inferer, Cropper

# setup device and checkpoint
device = 'cuda:0'

checkpoint_path = "/Vol1/dbstore/datasets/k.iskakov/share/face_expression/gold_checkpoints/siamese+mouth+keypoints_3d_loss+expression_loss/checkpoint_000019.pth"
config_path = "/Vol1/dbstore/datasets/k.iskakov/share/face_expression/gold_checkpoints/siamese+mouth+keypoints_3d_loss+expression_loss/config.yaml"

inferer = Inferer(
    config_path,
    checkpoint_path,
    device='cuda:0'
)
    
cropper = Cropper(
    rvec,
    tvec,
    K_d_rgb,
    dist=d_rgb
)

In [ ]:
count = 0
times = []
for fid in tqdm(frame_ids):
    if fid not in joints_poses_dict:
        continue
        
    joints_poses = joints_poses_dict[fid]

    # load rgb image   
    img_path = root_folder_rgb + '/' + pid_lbl + '/' + dev_lbl + '/color_undistorted/' + str(fid).zfill(6) + '.jpg'
    img = cv2.imread(img_path)[:, :, ::-1]

    im_size = (img.shape[1], img.shape[0])

    if img is None:
        print('no rgb frame')
    
    ### (!) prediction
#     for i in range(1000):
    start_time = time.time()
    img_cropped, face_bbox = cropper.forward(img, joints_poses)
    expression_pred, jaw_pose_pred, keypoints_2d = inferer.forward(img_cropped, beta=None)
    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
    mean_time = np.mean(times)
    print(f"Mean time/FPS: {mean_time:.3f}/{1/mean_time:.1f}")
    
    # infer SMPLX
    part_id = -1
    rot_t, trans_t, pose_body_t, pose_hand_t, pose_jaw_t, pose_eye_t, betas_t, expr_t, is_male, is_vposer = \
        get_smplx_params(fit_dataset_path, dev_lbl, fit_type, pid_lbl, part_id, fid, vposer_ckpt)
    
    exp_bm = init_smplx_inf_model(pk_root, is_male, fit_type)
    
    faces_np = exp_bm.f.detach().cpu().numpy()
    main_renderer = init_renderer(faces_np)
    
    # GT
    body = exp_bm(rot_t, pose_body_t, pose_hand_t, pose_jaw_t, pose_eye_t, betas_t, trans_t, expr_t)
    verts = body.v[0].detach().cpu().numpy()

    main_renderer.render(verts, np.eye(3), K=K_rgb)
    gt_cimg = main_renderer.get_aligned_camera_image()[:, :, :3]
    
    # pred
    expression_pred_t = torch.from_numpy(expression_pred).unsqueeze(0)
    jaw_pose_pred_t = torch.from_numpy(jaw_pose_pred).unsqueeze(0)
    
    body_pred = exp_bm(rot_t, pose_body_t, pose_hand_t, jaw_pose_pred_t, pose_eye_t, betas_t, trans_t, expression_pred_t)
    verts_pred = body_pred.v[0].detach().cpu().numpy()

    main_renderer.render(verts_pred, np.eye(3), K=K_rgb)
    pred_cimg = main_renderer.get_aligned_camera_image()[:, :, :3]
    
    # crop
    img_croppped = utils.common.crop_image(img, face_bbox)
    keypoints_2d -= face_bbox[:2]
    img_croppped = utils.vis.vis_image_with_keypoints_2d_numpy(keypoints_2d, img_croppped, 1)
    
    cimgs_cropped = [utils.common.crop_image(x, face_bbox) for x in (pred_cimg, gt_cimg)]

    cobmibned_img = np.concatenate([img_croppped, cimgs_cropped[0], cimgs_cropped[1]], axis=1)
    if count % 50 == 0:
        plt.imshow(cobmibned_img)
        plt.show()

    img_path = os.path.join(frame_dir, f"{count:06d}.jpg")
    cv2.imwrite(img_path, cobmibned_img[:, :, ::-1])

    count += 1

In [ ]:
cmd = [
    "ffmpeg",
    "-y",
    "-framerate", "25",
    "-i", os.path.join(frame_dir, "%06d.jpg"),
    "-c:v", "libx264",
    "-vf", "fps=25",
    "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2",
    "-pix_fmt", "yuv420p",
    output_video_path
]

result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if result.returncode:
    raise ValueError(result.stderr.decode("utf-8"))

In [ ]:
print(f"Output video path:\n{output_video_path}")

---